# Import & Dataset 읽어오기

In [1]:
from datasets import concatenate_datasets, load_from_disk
from src.retriever.retrieval.sparse_retrieval import SparseRetrieval
from transformers import AutoTokenizer

org_dataset = load_from_disk('data/train_dataset')
print(org_dataset)
full_ds = concatenate_datasets(
        [
            org_dataset["train"].flatten_indices(),
            org_dataset["validation"].flatten_indices(),
        ]
    )  # train dev 를 합친 4192 개 질문에 대해 모두 테스트
print("*" * 40, "query dataset", "*" * 40)
print(full_ds)

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})
**************************************** query dataset ****************************************
Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 4192
})


# Retriever 선언 및 Retriever 진행
* 아마 초기엔 학습 진행하는데 시간 좀 걸릴예정
    * 약 15분정도 소요(승범 컴 기준)
    * 노션에 mrc/검증결과/Sparse 처리에 가장 좋은 bm25올려났으므로 그거 data 폴더 안에 넣으면 실행가능.

In [4]:
# 위에서 선언한거 가져오기 Retriever
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased", use_fast=False,)
retriever = SparseRetrieval(
    tokenize_fn=tokenizer.tokenize,
    data_path="./data/",
    context_path="wikipedia_documents.json",
    mode = "bm25",
    max_feature=1000000,
    ngram_range=(1,2),
    #tokenized_docs = tokenized_docs,
)
retriever.get_sparse_embedding()

# Top k 조절.
df = retriever.retrieve(full_ds, topk=10)

# 여기서 df가 reader가 사용할 데이터 프레임입니다.
df.head(5)

Lengths of unique contexts : 56737
Building bm25 embedding...
Start Initializing...


Tokenizing...: 100%|██████████| 56737/56737 [02:20<00:00, 405.09it/s]


Generating n-grams and building vocabulary...


Generating n-grams: 100%|██████████| 56737/56737 [00:35<00:00, 1590.61it/s]


Current mode : bm25
End Initialization


Calculating BM25:   7%|▋         | 4/57 [01:51<23:25, 26.51s/it]

* question : 질문
* id : mrc 번호
* retrieval_context : retrieval 결과로 나온 topk를 하나의 text로 만든것
* original_context : 실제 retrieval 정답
* answers: dict 형태
    * answers_start : original_context에서 어디에서 시작하는지
    * text: 정답의 text값이 뭔지 

In [ ]:
len(df['retrieval_context'][0])

13059

## Retriever 결과물 DatasetDict로 저장
reader는 데이터셋을 DatasetDict형태로 받으므로, df에서 변환이 필요하다

In [ ]:
from datasets import Dataset, DatasetDict

# df를 Dataset으로
retriever_dataset = Dataset.from_pandas(df)

# DatasetDict로 변환 (train과 validation을 동일 데이터로 초기화)
dataset_dict = DatasetDict({
    'train': retriever_dataset,
    'validation': retriever_dataset
})

# train 스플릿: 'original_context'를 'context'로 사용
dataset_dict['train'] = dataset_dict['train'].remove_columns(['retrieval_context']).rename_column('original_context', 'context')

# validation 스플릿: 'retrieval_context'를 'context'로 사용
dataset_dict['validation'] = dataset_dict['validation'].remove_columns(['original_context']).rename_column('retrieval_context', 'context')

print(dataset_dict)


NameError: name 'df' is not defined

In [ ]:
# 별도 파일로 저장
dataset_dict.save_to_disk('outputs/bm25')

Saving the dataset (0/1 shards):   0%|          | 0/4192 [00:00<?, ? examples/s]

# Reader

In [ ]:
from transformers import HfArgumentParser
from transformers import TrainingArguments
from datasets import DatasetDict

from src.reader.model.reader import Reader
from src.utils.arguments import DataTrainingArguments, ModelArguments

#args 로드
model_args = ModelArguments()
data_args =  DataTrainingArguments()
training_args = TrainingArguments(output_dir='outputs')

# 세부 세팅
# model_args.model_name_or_path = <모델경로/huggingface>

training_args.do_train =True
training_args.do_eval = True

# 디스크로 데이터셋 불러오기
# dataset = DatasetDict.load_from_disk('outputs/bm25')

#데이터셋 바로 받아올 경우
dataset = dataset_dict

# output_dir은 training_args에 설정된 값을 사용
print("Output directory:", training_args.output_dir)



reader_model = Reader(model_args, data_args, training_args, dataset)
reader_model.run()




Output directory: outputs


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: --do_train requires a train dataset